In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_nyc_salary = pd.read_csv('nyc_salary.csv')
df_skill_list = pd.read_csv('skill_cell_nyc.csv', header=None)

In [ ]:
df_update = pd.concat([df_nyc_salary, df_skill_list], axis=1).drop(columns='desc')

In [ ]:
df_update.columns = ['company', 'rating', 'title', 'location', 'min','max','datepost','skills']

In [ ]:
df_update.describe()

In [ ]:
import plotly.express as px
tips = df_update.dropna()
fig = px.histogram(tips, x="rating",
                   marginal="violin", # or violin, rug
                   hover_data=tips.columns)
fig.show()

In [ ]:
df_update['rating']

In [ ]:
df_noskill = df_update.drop(['skills'], axis=1)
df_skill = df_update['skills'].str.get_dummies(sep=',')
df_skill_rank = df_skill.sum().to_frame(name='count').sort_values(by=['count'], ascending=False)[:50]
df_skill_rank.sort_values(by=['count'], ascending=True).plot.barh(figsize=(20,20))

In [ ]:
df_update_all_num = pd.concat([df_update, df_skill], axis=1)
df_update_all_num = df_update_all_num.drop(columns=['company','title','location','skills','datepost'])
df_update_all_num

In [ ]:
df_update_all_num.columns

In [ ]:
df_update_all_num = df_update_all_num.sort_values('max', ascending=False)

In [ ]:
df_update_all_num

In [ ]:
df_update_all_num = df_update_all_num.reset_index(drop=True)
df_update_all_num.head()


In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [ ]:
feature_names = list(df_update_all_num.drop(columns=['min','max']).columns)

In [ ]:
def max_n_estimators(start_n_estimators,max_n_estimators, step):
    RMSE_list = []
    n_estimators_list = []

    X = df_update_all_num[feature_names].values
    y = df_update_all_num['max'].values

    data_dmatrix=xgb.DMatrix(data=X, label=y)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)


    for n_estimators in np.arange(start_n_estimators,max_n_estimators, step):
        xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42,n_estimators = n_estimators)

        xgb_model.fit(X_train, y_train)

        y_pred = xgb_model.predict(X)

        mse=mean_squared_error(y, y_pred)
        n_estimators_list.append(n_estimators)
        RMSE_list.append(np.sqrt(mse))
    df = pd.DataFrame({'n_estimators_list' : n_estimators_list,
                           'RMSE_list' : RMSE_list},
                          columns=['n_estimators_list','RMSE_list'])
    return df

In [ ]:
df_nestimators = max_n_estimators(50,5000,50)

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df_nestimators['n_estimators_list'][1:], y=df_nestimators['RMSE_list'][1:],
                    mode='markers',
                    name='n_estimators'))
fig.show()

In [ ]:
X = df_update_all_num[feature_names]
y = df_update_all_num['max']

data_dmatrix=xgb.DMatrix(data=X, label=y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

xgb_model_max = xgb.XGBRegressor(objective="reg:squarederror", random_state=123,n_estimators = 1700)

xgb_model_max.fit(X_train, y_train)

y_pred_max = xgb_model_max.predict(X)

mse=mean_squared_error(y, y_pred_max)

print('RMSE:',np.sqrt(mse))

In [ ]:
X = df_update_all_num[feature_names]
y = df_update_all_num['min']

data_dmatrix=xgb.DMatrix(data=X, label=y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

xgb_model_min = xgb.XGBRegressor(objective="reg:squarederror", random_state=123,n_estimators = 1700)

xgb_model_min.fit(X_train, y_train)

y_pred_min = xgb_model_min.predict(X)

mse=mean_squared_error(y, y_pred_min)

print('RMSE:',np.sqrt(mse))

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xgb_model_min,num_trees=20)
plt.rcParams['figure.figsize'] = [20, 20]
plt.show()

In [ ]:
y_pred_com = pd.DataFrame(
    {'y_min': df_update_all_num['min'],
     'y_max': df_update_all_num['max'],
     'y_pred_min': y_pred_min,
     'y_pred_max': y_pred_max,
    })
y_pred_com

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=np.arange(0,496), y=y_pred_com['y_pred_min'],
                    mode='markers',
                    name='y_pred_min'))
fig.add_trace(go.Scatter(x=np.arange(0,496), y=y_pred_com['y_pred_max'],
                    mode='markers',
                    name='y_pred_max'))
fig.add_trace(go.Scatter(x=np.arange(0,496), y=y_pred_com['y_min'],
                    mode='markers',
                    name='y_min'))
fig.add_trace(go.Scatter(x=np.arange(0,496), y=y_pred_com['y_max'],
                    mode='markers',
                    name='y_max'))
fig.show()

In [ ]:
import pickle
pickle.dump(xgb_model_min, open('ny_min_xgb_model_min.pickle', 'wb'))
pickle.dump(xgb_model_max, open('ny_max_xgb_model_min.pickle', 'wb'))

In [ ]:
xgb_model_min_loaded = pickle.load(open('ny_min_xgb_model_min.pickle', "rb"))
xgb_model_max_loaded = pickle.load(open('ny_max_xgb_model_min.pickle', "rb"))
import os
print(round(os.path.getsize('ny_max_xgb_model_min.pickle')/1000000,1), 'MB')

In [ ]:
print('Predicted Salary Range($): ',int(xgb_model_min_loaded.predict(X[:1])),'~',int(xgb_model_max_loaded.predict(X[:1])))

In [ ]:
def NYC_salary_with_skills(rating,inputs_list):
    sample_list = [0] * (len(X.columns))
    input_X = pd.DataFrame([sample_list],
                 columns =list(X.columns))

    for inputs in inputs_list:
        if inputs in list(input_X.columns):
            input_X[inputs] = 1
    input_X['rating'] = rating
    salary = (int(xgb_model_min_loaded.predict(input_X[:1])),int(xgb_model_max_loaded.predict(input_X[:1])))
    return salary

In [ ]:
NYC_salary_with_skills(3.6,['Python','SQL','Tableau','R','Microsoft-Office'])

In [ ]:
skill_money = {}
skill_money_list = []
for skill in list(X.columns)[1:]:
    skill_money['skill'] = skill
    skill_money['salary'] = NYC_salary_with_skills(3.6,[skill])
    skill_money_list.append(skill_money)
    skill_money = {}
single_skill = pd.DataFrame(skill_money_list).sort_values('salary',ascending=False)
single_skill

In [ ]:
from itertools import combinations
L = list(X.columns)[1:]
unique_skill_pair = ()
for group_num in np.arange(1,5,1):
    unique_skill_pair += tuple([",".join(map(str, comb)) for comb in combinations(L, group_num)])

skill_money = {}
skill_money_list = []
for skill in unique_skill_pair:
    skill_money['skill'] = skill
    skill_money['salary'] = NYC_salary_with_skills(3.6,[skill])
    skill_money_list.append(skill_money)
    skill_money = {}
df_skill_pair = pd.DataFrame(skill_money_list).drop_duplicates().sort_values('salary',ascending=False)

In [ ]:
top_skill_salary = df_skill_pair[:5].reset_index(drop=True)
top_skill_salary

In [ ]:
single_skill[:10]

In [ ]:
salary_skills_list = ['Python','SQL','Tableau']

In [ ]:
suggest_list = []
all_list = salary_skills_list

for skill in single_skill['skill'][:5]:
    if skill not in salary_skills_list:
        suggest_list.append(skill)

    suggest_list_salary = {}
    suggest_list_salary_list = []
    for skill in suggest_list:
        all_list = []
        all_list = salary_skills_list+ [skill]
        suggest_list_salary['skill'] = skill
        suggest_list_salary['salary'] = int(np.subtract(NYC_salary_with_skills(3.6,all_list), NYC_salary_with_skills(3.6,salary_skills_list)).mean())
        suggest_list_salary_list.append(suggest_list_salary)
        suggest_list_salary = {}
        
    suggest_skills = pd.DataFrame(suggest_list_salary_list).sort_values('salary',ascending=False)
suggest_skills = suggest_skills[suggest_skills['salary'] > 0 ]
suggest_skills.columns = ['skill', 'with such more salary']
suggest_skills

### Recommending top 3 Skills with salary increase & Adding Rating to Function

In [ ]:
skill_money = {}
skill_money_list = []
for skill in list(X.columns)[1:]:
    skill_money['skill'] = skill
    skill_money['salary'] = NYC_salary_with_skills(3.6,[skill])
    skill_money_list.append(skill_money)
    skill_money = {}
single_skill = pd.DataFrame(skill_money_list).sort_values('salary',ascending=False)
single_skill

In [ ]:
import pickle
import pandas as pd
import numpy as np
xgb_model_min_loaded = pickle.load(open('ny_min_xgb_model_min.pickle', "rb"))
xgb_model_max_loaded = pickle.load(open('ny_max_xgb_model_min.pickle', "rb"))


sel_features = ['rating', 'AI', 'AWS', 'Azure', 'Big-Data', 'C/C++', 'Data-Analysis',
       'Data-Warehouse', 'Hadoop', 'Hive', 'Java', 'Kafka', 'Linux', 'MATLAB',
       'Machine-Learning', 'Microsoft-Office', 'Microsoft-SQL-Server',
       'Natural-Language-Processing', 'NoSQL', 'Oracle', 'Pig', 'Python', 'R',
       'SAS', 'SQL', 'Scala', 'Scripting', 'Spark', 'Tableau', 'TensorFlow']

def NYC_salary_with_skills(rating,inputs_list):
    sample_list = [0] * (len(sel_features))
    input_X = pd.DataFrame([sample_list],
                 columns =list(sel_features))

    for inputs in inputs_list:
        if inputs in list(input_X.columns):
            input_X[inputs] = 1
    input_X['rating'] = rating
    salary = (int(xgb_model_min_loaded.predict(input_X[:1])),int(xgb_model_max_loaded.predict(input_X[:1])))
    return salary

def NYC_salary_with_skills_and(rating, inputs_list):
    
    sample_list = [0] * (len(sel_features))
    input_X = pd.DataFrame([sample_list],
                 columns =list(sel_features))

    for inputs in inputs_list:
        if inputs in list(input_X.columns):
            input_X[inputs] = 1
    salary = (int(xgb_model_min_loaded.predict(input_X[:1])),int(xgb_model_max_loaded.predict(input_X[:1])))

    
    # suggest skill with more salary    
    suggest_list = []
    all_list = inputs_list

    for skill in single_skill['skill'][:10]:
        if skill not in inputs_list:
            suggest_list.append(skill)

        suggest_list_salary = {}
        suggest_list_salary_list = []
        for skill in suggest_list:
            all_list = []
            all_list = inputs_list+ [skill]
            suggest_list_salary['skill'] = skill
            suggest_list_salary['salary'] = int(np.subtract(NYC_salary_with_skills(rating,all_list), NYC_salary_with_skills(rating,inputs_list)).mean())
            suggest_list_salary_list.append(suggest_list_salary)
            suggest_list_salary = {}
    suggest_skills = pd.DataFrame(suggest_list_salary_list).sort_values('salary',ascending=False)
    suggest_skills = suggest_skills[suggest_skills['salary'] > 0 ]
    suggest_skills.columns = ['Skill', 'Salary increase by($)']
    suggest_skills    
    print('Annual Salary Range($):',salary)
    return suggest_skills[:3]

skill_money = {}
skill_money_list = []
for skill in sel_features[1:]:
    skill_money['skill'] = skill
    skill_money['salary'] = NYC_salary_with_skills(3.6,[skill])
    skill_money_list.append(skill_money)
    skill_money = {}
single_skill = pd.DataFrame(skill_money_list).sort_values('salary',ascending=False)

In [ ]:
NYC_salary_with_skills_and(3.6,['Python','R','Tableau'])